In [20]:
import pandas as pd

Memuat Dataset

In [21]:
df = pd.read_csv('layoffs_1.csv')

In [3]:
# Diasumsikan `world_layoffs_df` adalah DataFrame yang berisi data dari `world_layoffs.layoffs`
world_layoffs_df = pd.read_csv('layoffs_1.csv')

# A. MEMERIKSA DAN MENGHAPUS DATA DUPLIKAT

In [4]:
# 1. Membuat DataFrame sementara (setara dengan membuat tabel sementara di SQL)
layoffs_staging = world_layoffs_df.copy()

In [5]:
# 2. Mengidentifikasi dan menghapus data duplikat

# Memeriksa duplikat
duplikat = layoffs_staging[layoffs_staging.duplicated(subset=['company', 'industry', 'total_laid_off', 'date'], keep=False)]
print("Duplikat yang ditemukan:\n", duplikat)

Duplikat yang ditemukan:
                company       location        industry  total_laid_off  \
118              Yahoo    SF Bay Area        Consumer          1600.0   
625   Wildlife Studios      Sao Paulo        Consumer           300.0   
816                Oda           Oslo            Food            70.0   
817                Oda           Oslo            Food            70.0   
818                Oda           Oslo            Food            70.0   
1491             Cazoo         London  Transportation           750.0   
1492             Cazoo         London  Transportation           750.0   
1551          Terminus        Atlanta       Marketing             NaN   
1552          Terminus        Atlanta       Marketing             NaN   
1689            Casper  New York City          Retail             NaN   
2254             Hibob       Tel Aviv              HR            70.0   
2357             Yahoo    SF Bay Area        Consumer          1600.0   
2358             Hibob   

In [6]:
# Menghapus duplikat (menyimpan kejadian pertama)
layoffs_staging = layoffs_staging.drop_duplicates(subset=['company', 'industry', 'total_laid_off', 'date'], keep='first')

In [7]:
# Memverifikasi penghapusan duplikat
print("Data setelah menghapus duplikat:\n", layoffs_staging)

Data setelah menghapus duplikat:
             company        location        industry  total_laid_off  \
0         Atlassian          Sydney           Other           500.0   
1          SiriusXM   New York City           Media           475.0   
2            Alerzo          Ibadan          Retail           400.0   
3            UpGrad          Mumbai       Education           120.0   
4              Loft       Sao Paulo     Real Estate           340.0   
...             ...             ...             ...             ...   
2352   HopSkipDrive     Los Angeles  Transportation             8.0   
2353    Panda Squad     SF Bay Area        Consumer             6.0   
2354  Tamara Mellon     Los Angeles          Retail            20.0   
2355       EasyPost  Salt Lake City       Logistics            75.0   
2356      Blackbaud      Charleston           Other           500.0   

      percentage_laid_off       date     stage        country  \
0                    0.05   3/6/2023  Post-IPO  

# B. MENSTANDARKAN DATA


In [8]:
# Menangani nilai yang hilang dan tidak valid di kolom 'industry'

# Mengidentifikasi nilai unik di kolom 'industry'
industri_unik = layoffs_staging['industry'].unique()
print("Industri unik:\n", industri_unik)

Industri unik:
 ['Other' 'Media' 'Retail' 'Education' 'Real Estate' 'Transportation'
 'Marketing' nan 'Healthcare' 'Security' 'Food' 'Fitness' 'Consumer'
 'Logistics' 'HR' 'Support' 'Travel' 'Crypto' 'Finance' 'Data' 'Sales'
 'Infrastructure' 'Hardware' 'Product' 'Construction' 'Legal' 'Energy'
 'Manufacturing' 'Recruiting' 'Aerospace' 'Crypto Currency' 'Fin-Tech'
 'CryptoCurrency']


In [9]:
# Menemukan baris dengan 'industry' NULL atau kosong
industri_hilang = layoffs_staging[layoffs_staging['industry'].isnull() | (layoffs_staging['industry'] == '')]
print("Industri yang hilang:\n", industri_hilang)

Industri yang hilang:
                   company     location industry  total_laid_off  \
8                  Airbnb  SF Bay Area      NaN            30.0   
330   Bally's Interactive   Providence      NaN             NaN   
736                  Juul  SF Bay Area      NaN           400.0   
1595              Carvana      Phoenix      NaN          2500.0   

      percentage_laid_off        date     stage        country  \
8                     NaN    3/3/2023  Post-IPO  United States   
330                  0.15   1/18/2023  Post-IPO  United States   
736                  0.30  11/10/2022   Unknown  United States   
1595                 0.12   5/10/2022  Post-IPO  United States   

      funds_raised_millions  
8                    6400.0  
330                   946.0  
736                  1500.0  
1595                 1600.0  


In [10]:
# Mengganti string kosong dengan NULL untuk manipulasi yang lebih mudah
layoffs_staging['industry'].replace('', pd.NA, inplace=True)

C:\Users\restu firmansyah\AppData\Local\Temp\ipykernel_11108\3916517896.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  layoffs_staging['industry'].replace('', pd.NA, inplace=True)


In [11]:
# Mengisi nilai 'industry' yang hilang berdasarkan nama perusahaan
layoffs_staging['industry'] = layoffs_staging.groupby('company')['industry'].transform(lambda x: x.ffill().bfill())

C:\Users\restu firmansyah\AppData\Local\Temp\ipykernel_11108\3914976511.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  layoffs_staging['industry'] = layoffs_staging.groupby('company')['industry'].transform(lambda x: x.ffill().bfill())


In [12]:
# Memverifikasi jika masih ada nilai 'industry' yang hilang
print("Baris dengan 'industry' yang hilang setelah pengisian:\n", layoffs_staging[layoffs_staging['industry'].isnull()])

Baris dengan 'industry' yang hilang setelah pengisian:
                  company    location industry  total_laid_off  \
330  Bally's Interactive  Providence      NaN             NaN   

     percentage_laid_off       date     stage        country  \
330                 0.15  1/18/2023  Post-IPO  United States   

     funds_raised_millions  
330                  946.0  


In [13]:
# Menstandarkan nilai di kolom 'industry'
layoffs_staging['industry'].replace({'Crypto Currency': 'Crypto', 'CryptoCurrency': 'Crypto'}, inplace=True)


C:\Users\restu firmansyah\AppData\Local\Temp\ipykernel_11108\3854007030.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  layoffs_staging['industry'].replace({'Crypto Currency': 'Crypto', 'CryptoCurrency': 'Crypto'}, inplace=True)


In [14]:

# Memverifikasi nilai unik di 'industry' setelah penstandaran
print("Industri unik setelah penstandaran:\n", layoffs_staging['industry'].unique())

Industri unik setelah penstandaran:
 ['Other' 'Media' 'Retail' 'Education' 'Real Estate' 'Transportation'
 'Marketing' 'Travel' 'Healthcare' 'Security' 'Food' 'Fitness' 'Consumer'
 'Logistics' 'HR' 'Support' 'Crypto' 'Finance' 'Data' 'Sales'
 'Infrastructure' 'Hardware' 'Product' 'Construction' 'Legal' 'Energy' nan
 'Manufacturing' 'Recruiting' 'Aerospace' 'Fin-Tech']


In [15]:

# Menstandarkan nilai di kolom 'country'
# Menghapus titik di akhir nama 'country'
layoffs_staging['country'] = layoffs_staging['country'].str.rstrip('.')

In [16]:
# Memverifikasi nilai unik di 'country' setelah penstandaran
print("Negara unik setelah penstandaran:\n", layoffs_staging['country'].unique())

Negara unik setelah penstandaran:
 ['Australia' 'United States' 'Nigeria' 'India' 'Brazil' 'France' 'Germany'
 'Israel' 'Sweden' 'United Kingdom' 'Japan' 'South Korea' 'China' 'Italy'
 'Singapore' 'Indonesia' 'Estonia' 'Canada' 'Ireland' 'Finland'
 'Netherlands' 'Spain' 'Portugal' 'Chile' 'Colombia' 'Argentina'
 'Seychelles' 'Austria' 'Mexico' 'Switzerland' 'Egypt' 'Kenya'
 'Luxembourg' 'Greece' 'Poland' 'Norway' 'Belgium' 'Denmark' 'Hong Kong'
 'New Zealand' 'Malaysia' 'Hungary' 'Vietnam' 'Thailand' 'Lithuania'
 'Ghana' 'Senegal' 'Pakistan' 'United Arab Emirates' 'Peru' 'Bahrain'
 'Romania' 'Turkey' 'Russia' 'Uruguay' 'Bulgaria' 'South Africa'
 'Czech Republic' 'Myanmar']


# C. MEMERIKSA NILAI NULL

In [17]:
# Memeriksa nilai NULL di 'total_laid_off'
total_laid_off_null = layoffs_staging[layoffs_staging['total_laid_off'].isnull()]
print("Baris dengan 'total_laid_off' NULL:\n", total_laid_off_null)


Baris dengan 'total_laid_off' NULL:
             company       location    industry  total_laid_off  \
9          Accolade        Seattle  Healthcare             NaN   
10           Indigo         Boston       Other             NaN   
16         Flipkart      Bengaluru      Retail             NaN   
17          Kandela    Los Angeles    Consumer             NaN   
18    Truckstop.com          Boise   Logistics             NaN   
...             ...            ...         ...             ...   
2340         Vacasa       Portland      Travel             NaN   
2345      Anyvision       Tel Aviv    Security             NaN   
2346          Popin  New York City     Fitness             NaN   
2347  Tuft & Needle        Phoenix      Retail             NaN   
2351        Service    Los Angeles      Travel             NaN   

      percentage_laid_off       date     stage        country  \
9                     NaN   3/3/2023  Post-IPO  United States   
10                    NaN   3/3/2023  Se

In [18]:
# Memeriksa baris di mana 'total_laid_off' dan 'percentage_laid_off' keduanya NULL
total_dan_persentase_null = layoffs_staging[layoffs_staging['total_laid_off'].isnull() & layoffs_staging['percentage_laid_off'].isnull()]
print("Baris dengan 'total_laid_off' dan 'percentage_laid_off' NULL:\n", total_dan_persentase_null)

Baris dengan 'total_laid_off' dan 'percentage_laid_off' NULL:
             company     location        industry  total_laid_off  \
9          Accolade      Seattle      Healthcare             NaN   
10           Indigo       Boston           Other             NaN   
16         Flipkart    Bengaluru          Retail             NaN   
18    Truckstop.com        Boise       Logistics             NaN   
57    Arch Oncology    St. Louis      Healthcare             NaN   
...             ...          ...             ...             ...   
2323       SpotHero      Chicago  Transportation             NaN   
2334          Zwift  Los Angeles         Fitness             NaN   
2340         Vacasa     Portland          Travel             NaN   
2345      Anyvision     Tel Aviv        Security             NaN   
2347  Tuft & Needle      Phoenix          Retail             NaN   

      percentage_laid_off       date     stage        country  \
9                     NaN   3/3/2023  Post-IPO  United 

In [22]:
# Simpan DataFrame df ke file bernama 'data_bersih.csv'
df.to_csv('DataCleaning.csv', index=False)
